In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [ ]:
# get data
data = pd.read_csv('Level_3c_NonPath_SP3_PsPu_PSL_1_2_3_435.csv') 

# split data
X=data.iloc[:,1:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

         Xc1.A      Xc1.R  ...  Grantham.Xd.29  Grantham.Xd.30
0    17.569047  10.980654  ...        0.033889        0.032943
1     7.805162   5.676482  ...        0.033243        0.031827
2     9.759306   7.617019  ...        0.032000        0.030636
3    13.897496   6.434026  ...        0.031503        0.032314
4    13.083343   6.413403  ...        0.031943        0.032270
..         ...        ...  ...             ...             ...
430   3.434023   2.113245  ...        0.029650        0.032397
431   8.597566   6.513308  ...        0.032343        0.028946
432   8.597410   1.719482  ...        0.033358        0.028789
433   5.471856   0.521129  ...        0.032696        0.027279
434   5.471856   0.521129  ...        0.032696        0.027279

[435 rows x 150 columns]
0      1
1      1
2      1
3      1
4      1
      ..
430    3
431    3
432    3
433    3
434    3
Name: Class, Length: 435, dtype: category
Categories (3, int64): [1, 2, 3]


In [ ]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [ ]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.20,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(150, 3 - 1) = 2 components.
  ChangedBehaviorWarn

In [ ]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(3 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 150)            91200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 128)            142848    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 283,651
Trainable params: 283,651
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/20
20/20 [==============================] - 36s 174ms/step - loss: 0.6993 - accuracy: 0.3259 - val_loss: 0.6332 - val_accuracy: 0.5072
Epoch 2/20
20/20 [==============================] - 0s 23ms/step - loss: 0.6304 - accuracy: 0.3880 - val_loss: 0.6192 - val_accuracy: 0.4737
Epoch 3/20
20/20 [==============================] - 0s 22ms/step - loss: 0.6024 - accuracy: 0.5975 - val_loss: 0.5354 - val_accuracy: 0.8469
Epoch 4/20
20/20 [==============================] - 0s 23ms/step - loss: 0.5239 - accuracy: 0.7406 - val_loss: 0.4062 - val_accuracy: 0.9474
Epoch 5/20
20/20 [==============================] - 0s 23ms/step - loss: 0.4291 - accuracy: 0.8077 - val_loss: 0.2861 - val_accuracy: 0.9904
Epoch 6/20
20/20 [==============================] - 0s 21ms/step - loss: 0.3338 - accuracy: 0.8570 - val_loss: 0.2013 - val_accuracy: 0.9856
Epoch 7/20
20/20 [==============================] - 0s 22ms/step - loss: 0.2525 - accuracy: 0.9102 - val_loss: 0.1477 - val_accuracy: 0.9856
Epoch 8/20


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.9696
test-acc = 0.9904306220095693
train-CFR =               precision    recall  f1-score   support

           0       0.96      0.96      0.96       202
           1       1.00      0.95      0.98       221
           2       0.95      1.00      0.98       202

    accuracy                           0.97       625
   macro avg       0.97      0.97      0.97       625
weighted avg       0.97      0.97      0.97       625

test-CFR =               precision    recall  f1-score   support

           0       0.97      1.00      0.99        76
           1       1.00      0.96      0.98        57
           2       1.00      1.00      1.00        76

    accuracy                           0.99       209
   macro avg       0.99      0.99      0.99       209
weighted avg       0.99      0.99      0.99       209

